In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

In [2]:
def deleteId(ds):
    return ds.drop(columns='Id')

def deleteNaN(train_ds, test_ds, critval):
    fullsize = train_ds.shape[0]
    new_train = train_ds.copy()
    new_test = test_ds.copy()
    for feature in new_train.columns:
        nulls = new_train[feature].isnull().sum()
        percent = nulls / fullsize
        if (percent > critval):
            new_train = new_train.drop(columns=feature)
            new_test = new_test.drop(columns=feature)
    return new_train, new_test

def convertToNumeric(train_ds, test_ds):
    new_train = train_ds.copy()
    new_test = test_ds.copy()
    LE = LabelEncoder()
    for feature in new_train.columns[:-1]:
        if (new_train[feature].dtype == 'object'):
            new_train[feature] = LE.fit_transform(new_train[feature])
            new_test[feature] = LE.fit_transform(new_test[feature])
    return new_train, new_test

def check_error(preds, gt):
    print('RMSE Error:', mean_squared_error(np.log(preds), np.log(gt), squared=False))

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train = train.drop_duplicates()

train = deleteId(train)  
train, test = deleteNaN(train, test ,critval=0.7)
train, test = convertToNumeric(train, test)

y_train = train['SalePrice'].values
x_train = train.drop(columns='SalePrice').values
x_super_test = test.values

X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, random_state=98987)

imp = SimpleImputer(strategy='mean')
X_train = imp.fit_transform(X_train)
X_test = imp.fit_transform(X_test)

### PCA + Scaler


In [4]:
pca = PCA(n_components = 20)
X_train = pca.fit_transform(X_train, y_train)
X_test = pca.transform(X_test)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train, y_train)
X_test = scaler.transform(X_test)

### Random Forest

In [5]:
parameters = {
    'criterion':(['squared_error']), 
    'max_depth': (1000, 1500, 2000, None),
    'max_features':('auto', 'sqrt', 'log2'),
    'n_estimators':(5, 10, 20, 50),
    'min_samples_split':(2, 5, 9)}

model = RandomForestRegressor()
gs_model = GridSearchCV(model, parameters)
gs_model.fit(X_train[:300], y_train[:300])
print(f"Best parameters: {gs_model.best_params_}")

/home/mixail/Документы/phys/MachineLearning/venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/mixail/Документы/phys/MachineLearning/venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/mixail/Документы/phys/MachineLearning/venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explic

Best parameters: {'criterion': 'squared_error', 'max_depth': 1000, 'max_features': 'auto', 'min_samples_split': 9, 'n_estimators': 5}


/home/mixail/Документы/phys/MachineLearning/venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


In [6]:
model = RandomForestRegressor(**gs_model.best_params_)
model.fit(X_train, y_train)
check_error(model.predict(X_test), y_test)

RMSE Error: 0.20707699030367605


/home/mixail/Документы/phys/MachineLearning/venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


### XGBoost

In [7]:
parameters = {
    "learning_rate": (0.05, 0.1, 0.15),
    "max_depth": [4, 5, 6],
    "min_child_weight": [7, 9, 12],
    "n_estimators":(30,50, 70)}

model = XGBRegressor()
gs_model = GridSearchCV(model, parameters)
gs_model.fit(X_train[:300], y_train[:300])
print(f"Best parameters: {gs_model.best_params_}")

Best parameters: {'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 12, 'n_estimators': 70}


In [8]:
model = XGBRegressor(**gs_model.best_params_)
model.fit(X_train, y_train)
check_error(model.predict(X_test), y_test)

RMSE Error: 0.2029348814603594


В отличие от поуровневого (горизонтального) роста в XGBoost, LightGBM выполняет листовой (вертикальный) рост, что приводит к большему снижению потерь и, в свою очередь, к более высокой точности при более высокой скорости. Но это также может привести к переобучению обучающих данных, которые можно обработать с помощью параметра максимальной глубины, указывающего, где будет происходить разделение. Следовательно, XGBoost способен создавать более надежные модели, чем LightGBM.

In [9]:
model = LGBMRegressor()

parameters = {
    'num_leaves': [i for i in range(3,15,2)],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 4, 5, 6, 7],
    'n_estimators': [50, 100],}

gs_model = GridSearchCV(model, parameters)
gs_model.fit(X_train[:300], y_train[:300])
print(f"Best parameters: {gs_model.best_params_}")

Best parameters: {'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 50, 'num_leaves': 11}


In [10]:
model = LGBMRegressor(**gs_model.best_params_)
model.fit(X_train, y_train)
check_error(model.predict(X_test), y_test)

RMSE Error: 0.24040962990936537


In [11]:
def getCategorialFeatures(train_ds):
    cat_features = []
    iter = 0
    for feature in train_ds.columns[:-1]:
        if (train_ds[feature].dtype == 'object'):
            cat_features.append(iter)
        iter +=1 
    return cat_features

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train = train.drop_duplicates()

train = deleteId(train)  
train, test = deleteNaN(train, test ,critval=0.7)
cat_features = getCategorialFeatures(train)

y_train = train['SalePrice'].values
x_train = train.drop(columns='SalePrice').values
x_super_test = test.values

X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, random_state=98987)

imp = SimpleImputer(strategy='most_frequent')
X_train = imp.fit_transform(X_train)
X_test = imp.fit_transform(X_test)

# Cat Boost
1) В качестве базового алгоритма используются небрежные решающие деревья (ODT - Obvious Decision Tree)
2) В качестве метода ансаблированния используется модифицированный градиентный бустинг. Суть его следующая:
   * Перед обучением создаётся s перестановок обучающей выборки $X_1, X_2, ..., X_s$. 
   * Кадая из этих перестановок $X_\sigma$ делится на части, длина которых возрастает в геометрической прогрессии ($2^i$). 
   * Каждый новый алгоритм ансамбля $b_t$ строится на перстановке $X_\sigma$, где $\sigma$ выбирается случайным образом от 0 до s
   * $$b_t := \arg\min_b\sum_{i=1}^{l}{(b(x_i) + g_{ti})^2}$$
   * Градиент $g_{ti}$ вычисляется по выборке $X^{\sigma j}$, где j - это подвыборка перстановки в которой не учасвствовал объект $x_i$
   * $$g_{t i} = \mathcal{L}'(a^{r j}_{t-1}(x_i), y_i) $$
   * $$j = \log_2(i - 1)$$
3) Следующая фича - работа с категориальными признаками. Категориальные признаки преобразуются в вещественные с помощью метода статистики по целевому признаку (TS).


In [12]:
model = CatBoostRegressor(loss_function='RMSE', cat_features=cat_features)
model.fit(X_train, y_train, plot=True)
check_error(model.predict(X_test), y_test)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.042748
0:	learn: 76930.4091113	total: 57.1ms	remaining: 57.1s
1:	learn: 74859.8884342	total: 65.2ms	remaining: 32.5s
2:	learn: 73006.7045395	total: 73.8ms	remaining: 24.5s
3:	learn: 71005.1089006	total: 85.9ms	remaining: 21.4s
4:	learn: 69301.2682476	total: 92.6ms	remaining: 18.4s
5:	learn: 67572.8459105	total: 102ms	remaining: 17s
6:	learn: 65919.7517106	total: 108ms	remaining: 15.3s
7:	learn: 64315.7210541	total: 114ms	remaining: 14.1s
8:	learn: 62720.0306505	total: 121ms	remaining: 13.3s
9:	learn: 61306.0702768	total: 127ms	remaining: 12.6s
10:	learn: 59899.0116968	total: 139ms	remaining: 12.5s
11:	learn: 58540.3382105	total: 148ms	remaining: 12.1s
12:	learn: 57218.6861287	total: 156ms	remaining: 11.8s
13:	learn: 55974.9360650	total: 167ms	remaining: 11.7s
14:	learn: 54755.6045223	total: 174ms	remaining: 11.4s
15:	learn: 53554.8727241	total: 185ms	remaining: 11.4s
16:	learn: 52401.9591590	total: 195ms	remaining: 11.3s
17:	learn: 51280.5153216	total: 206ms	rema